<a href="https://colab.research.google.com/github/BrystofKlazek/RAD/blob/main/code/01RAD_Ex11_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 11 - Hands on

Authors: Your names here  
Date: 2025-12-10  

---

## Task Description

The dataset is based on the **House Sales in King County, USA** dataset, which can be found, for example, on kaggle.com or in the `moderndive` library under the name `house_prices`. The original dataset contains house sale prices in the King County area, which includes Seattle, and the data was collected between May 2014 and May 2015. For our purposes, several variables have been removed, and the dataset has been significantly reduced and slightly modified.

The dataset has already been split into three parts and modified, all of which will be used progressively throughout this assignment.

---

## Variables Description

The dataset contains the following 18 variables, and our goal is to explore the influence of 12 of them on the target variable `price`.

| Feature         | Description                                           |
|------------------|-------------------------------------------------------|
| `id`            | Unique identifier for a house                         |
| `price`         | Sale price (prediction target)                        |
| `bedrooms`      | Number of bedrooms                                    |
| `bathrooms`     | Number of bathrooms                                   |
| `sqft_living`   | Square footage of the home                            |
| `sqft_lot`      | Square footage of the lot                             |
| `floors`        | Total number of floors (levels) in the house          |
| `waterfront`    | Whether the house has a waterfront view               |
| `view`          | Number of times the house has been viewed             |
| `condition`     | Overall condition of the house                        |
| `grade`         | Overall grade given to the housing unit               |
| `sqft_above`    | Square footage of the house apart from the basement   |
| `sqft_basement` | Square footage of the basement                        |
| `yr_built`      | Year the house was built                              |
| `yr_renovated`  | Year when the house was renovated                     |
| `sqft_living15` | Living room area in 2015 (after renovations)          |
| `sqft_lot15`    | Lot size in 2015 (after renovations)                  |
| `split`         | Splitting variable with train, test, and validation samples |

---


In [ ]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt

# Load the dataset
url = "https://raw.githubusercontent.com/francji1/01RAD/main/data/01RAD_2024_house.csv"
house_rad = pd.read_csv(url)

# Display the first few rows of the dataset
house_rad.head()

# Convenience splits for later
train = house_rad.query('split == "train"').copy()
test  = house_rad.query('split == "test"').copy()
val   = house_rad.query('split == "validation"').copy()




---

## Exploratory and Graphical Analysis

### Question 1

Verify the dimensions of the dataset, the types of individual variables, and summarize the basic descriptive statistics of all variables. Plot a histogram and a density estimate for the target variable `price`. Can anything be inferred for future analysis?

---

### Question 2

Are all variables usable for analysis and prediction of house prices? If the data contains missing values (or strange or nonsensical observations), can they be imputed (corrected), or must they be removed from the dataset?

---

### Question 3

For the selected variables (`price`, `sqft_living`, `grade`, `yr_built`), verify whether the split into train, test, and validation datasets was random. That is, do these variables have approximately the same distributions across the train, test, and validation groups?

---


Consider to transform price (e.g. log‑price) and key size variables if distributions are highly skewed and compare residual diagnostics for transformed vs untransformed models later.

In [ ]:
# Dimensions
print(house_rad.shape)
print(house_rad.apply(len).unique()) #This comes out fine - all cols are the same length

print(house_rad.dtypes)

print(house_rad.describe().T)

print(house_rad['price'].describe())

low_price = house_rad[house_rad['price']==0]
print(low_price)

plt.figure()
plt.hist(house_rad['price'], bins=50)
plt.title("Price histogram")
plt.xlabel("price")
plt.ylabel("count")
plt.show()



In [ ]:
house_rad  = house_rad.drop(columns=['yr_renovated'])
house_rad = house_rad.drop(index=[148, 48])
house_rad = house_rad.query("grade <= 13").copy()


In [ ]:
house_rad['split'].value_counts()


In [ ]:
vars_q3 = ['price', 'sqft_living', 'grade', 'yr_built']

summary_by_split = (
    house_rad
    .groupby('split')[vars_q3]
    .describe()
    .round(2)
)

summary_by_split


In [ ]:
train = house_rad.query('split == "train"')
test  = house_rad.query('split == "test"')
val   = house_rad.query('split == "validation"')

for v in vars_q3:
    plt.figure()
    plt.hist(train[v].dropna(), bins=40, alpha=0.4, label='train')
    plt.hist(test[v].dropna(),  bins=40, alpha=0.4, label='test')
    plt.hist(val[v].dropna(),   bins=40, alpha=0.4, label='validation')
    plt.title(f"{v} distribution by split")
    plt.xlabel(v)
    plt.ylabel("count")
    plt.legend()
    plt.show()



## Linear Model (Use Only Training Data, i.e., split == "train")

### Question 4

Calculate the correlations between the regressors and visualize them. Also, compute the condition number (Kappa) and the variance inflation factor (VIF). If multicollinearity is present, decide which variables to use and justify your choices.

---

### Question 5

Using only the training data (split == "train") and all selected variables, find a suitable linear regression model that best predicts the target variable `price`, i.e., minimizes the mean squared error (MSE). Compare the VIF and Kappa values of the final model to those of the original regressor matrix.

---

### Question 6

For your selected model from the previous question, calculate the relevant influence measures. Provide the `id` for the top 20 observations with the highest values of DIFFITS and DFBetas, the highest leverage (hat values), and the highest Cook’s distance (i.e., 3 sets of 20 values). Which observations do you consider influential or outliers?

---

### Question 7

Validate the model graphically using residual plots (Residual vs. Fitted, QQ-plot, Cook’s distance, leverage, etc.). Based on this and the previous question, did you identify any suspicious observations in the data that might have resulted from dataset adjustments? Would you recommend removing these observations from the data?

---


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import patsy

train = house_rad.query('split == "train"').copy()
test  = house_rad.query('split == "test"').copy()
val   = house_rad.query('split == "validation"').copy()

exclude = {'price', 'split', 'id', 'Unnamed: 0'}
exclude = {c for c in exclude if c in train.columns}

num_cols = train.select_dtypes(include='number').columns
regressors = [c for c in num_cols if c not in exclude]

regressors

formula_full = "price ~ " + " + ".join(regressors)
formula_full



In [ ]:
corr_reg = train[regressors].corr()

corr_reg.round(3).head()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr_reg, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Correlation matrix of regressors (train)")
plt.show()



## Robust Approach

### Question 8

If you decided to remove any observations from the dataset, work with the filtered dataset, retrain the model from Question 5, and calculate the $R^2$ statistic and MSE on both the training and test data (split == "test").

---

### Question 9

Estimate the regression coefficients using a robust Total Least Squares (TLS), on both the filtered dataset (after removing any observations, if applicable) and the original full dataset. Compare the results and discuss any differences in the estimated coefficients and model performance. What insights can you draw about the impact of filtering observations on model robustness?


---
### Question 10

Select the final model and compare the MSE and $R^2$ on the training, test, and validation datasets. What do these values suggest about the quality of the model and potential overfitting? Is your model suitable for predicting house prices in the King County area? If so, does this prediction have any limitations?



---

## Machine Learning Approach


### Question 11
Apply machine learning-based linear regression methods, such as Ridge Regression, Lasso, or Elastic Net, to the dataset. Use the train-test split to evaluate model performance and focus on feature selection. Identify the most relevant features based on these methods and compare how the selected features impact the model's predictive performance. Discuss how regularization affects feature selection and the trade-offs between bias and variance in the context of house price prediction. Additionally, evaluate the stability of selected features across different methods and provide recommendations for choosing the optimal feature set.

Use
* Standardization of regressors (and possibly use of pipelines).
* Hyperparameter tuning via cross‑validation (e.g. grid search over λ/α).
* Compare OLS vs ridge vs lasso on the same train/test split.
